In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
file_path = 'data.xlsx'
sheet_name = 'Данные_Задание_2'
file_path = r'C:\Users\PC\Desktop\Софт\Tina\data.xlsx'
df = pd.read_excel(file_path, sheet_name=sheet_name)

In [3]:
df.head()

,Субъект РФ,OID МО,Наименование МО,Адрес,ID здания,Наименование здания,OID СП,Наименование СП,План,Факт
0,Алтайский край,1.2.643.5.1.13.13.12.2.22.1671,"КГБУЗ "" ТРОИЦКАЯ ЦРБ""",Алтайский Край п Гордеевский пер Цветочный 14,31279.0,Пролетарская СВА,1.2.643.5.1.13.13.12.2.22.1671.0.63681,Пролетарская СВА,1,1
1,Алтайский край,1.2.643.5.1.13.13.12.2.22.1671,"КГБУЗ "" ТРОИЦКАЯ ЦРБ""",Алтайский Край п Гордеевский пер Цветочный 14,31279.0,Пролетарская СВА,1.2.643.5.1.13.13.12.2.22.1671.0.663415,Пролетарская СВА СДП,1,0
2,Алтайский край,1.2.643.5.1.13.13.12.2.22.1671,"КГБУЗ "" ТРОИЦКАЯ ЦРБ""",Алтайский Край п Гордеевский пер Цветочный 14,31279.0,Пролетарская СВА,1.2.643.5.1.13.13.12.2.22.1671.0.333332,сдп Пролетарская СВА,1,0
3,Алтайский край,1.2.643.5.1.13.13.12.2.22.1671,"КГБУЗ "" ТРОИЦКАЯ ЦРБ""",Алтайский Край с Горновое ул Молодежная 24,69319.0,Горновской ФАП,1.2.643.5.1.13.13.12.2.22.1671.0.161740,Горновской ФАП,0,1
4,Алтайский край,1.2.643.5.1.13.13.12.2.22.1671,"КГБУЗ "" ТРОИЦКАЯ ЦРБ""",Алтайский Край с Еремино ул Школьная 19,69337.0,Ереминский ФАП,1.2.643.5.1.13.13.12.2.22.1671.0.161743,Ереминский ФАП,0,0


In [4]:
columns = {
    'Субъект РФ': 'Субъект РФ',
    'OID МО': 'OID МО',
    'Наименование МО': 'Наименование МО',
    'Адрес': 'Адрес',
    'ID здания': 'ID здания',
    'Наименование здания': 'Наименование здания',
    'OID СП': 'OID СП',
    'Наименование СП': 'Наименование СП',
    'План': 'План',
    'Факт': 'Факт'
}

In [5]:
df = df.drop_duplicates(subset=['OID МО', 'OID СП', 'ID здания'])

In [6]:
# Агрегация по ТВСП (OID МО + ID здания): План=1 если max(План)=1, аналогично для Факт
tvsp_grouped = df.groupby(['Субъект РФ', 'OID МО', 'Наименование МО', 'Адрес', 'ID здания', 'Наименование здания']).agg(
    plan_agg=('План', 'max'),
    fact_agg=('Факт', 'max')
).reset_index()

In [7]:
# Это и есть детализация по ТВСП: таблица в формате как для Показателя 1
detalization = tvsp_grouped.rename(columns={'plan_agg': 'План', 'fact_agg': 'Факт'})

In [8]:
detalization.to_excel('detalization.xlsx', index=False)

In [9]:
# Теперь расчет отчета по субъектам
# Группировка по субъектам РФ на основе агрегированных данных
grouped = tvsp_grouped.groupby('Субъект РФ').agg(
    plan_count=('plan_agg', 'sum'),  # Количество ТВСП где aggregated План == 1
    fact_in_plan=('fact_agg', lambda x: ((tvsp_grouped.loc[x.index, 'plan_agg'] == 1) & (x == 1)).sum()),  # Где План == 1 и Факт == 1
    fact_total=('fact_agg', 'sum')  # Всего где Факт == 1
)

In [10]:
# Расчет Показателя 2
grouped['Показатель 2'] = grouped['fact_in_plan'] / grouped['plan_count'].where(grouped['plan_count'] > 0, 0)

In [13]:
# Для РФ целиком
rf_total = pd.DataFrame({
    'plan_count': [tvsp_grouped['plan_agg'].sum()],
    'fact_in_plan': [((tvsp_grouped['plan_agg'] == 1) & (tvsp_grouped['fact_agg'] == 1)).sum()],
    'fact_total': [tvsp_grouped['fact_agg'].sum()]
}, index=['РФ'])
rf_total['Показатель 2'] = rf_total['fact_in_plan'] / rf_total['plan_count'].where(rf_total['plan_count'] > 0, 0)

In [19]:
# Объединение отчета
report = pd.concat([grouped, rf_total])

In [20]:
# Переименование колонок для отчета
report.columns = [
    'Количество плановых ТВСП по Требованию 2',
    'Количество фактических ТВСП, попадающих в план (факт от плана)',
    'Количество фактических ТВСП, соответствующих Требованию 2 (всего, безотносительно плана)',
    'Показатель 2'
]

In [21]:
# Добавление колонки Субъект РФ (как индекс)
report2 = report.reset_index().rename(columns={'index': 'Субъект РФ'})

In [22]:
report2.to_excel('report2.xlsx', index=False)